In [1]:
import datetime
import pandas as pd
import numpy as np
import yfinance as yf


# for plotting purposes
from datetime import datetime
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
# Equivalent timeframe mappings
frame_M1 = '1m'
frame_M15 = '15m'
frame_M30 = '30m'
frame_H1 = '1h'
frame_H4 = '4h'
frame_D1 = '1d'
frame_W1 = '1wk'


# Matching your MT5 asset list with Yahoo tickers
assets = {
    'EURUSD': 'EURUSD=X',
    'USDCHF': 'USDCHF=X',
    'GBPUSD': 'GBPUSD=X',
    'USDCAD': 'USDCAD=X',
    'BTCUSD': 'BTC-USD',
    'ETHUSD': 'ETH-USD',
    'XAUUSD': 'XAUUSD=X',
    'XAGUSD': 'XAGUSD=X',
    'SP500m': '^GSPC',
    'UK100': '^FTSE'
}

In [9]:
# SUPPORT AND RESISTANCE

def get_quotes(time_frame,  period='2yrs', asset='EURUSD'):
    try:
        symbol = assets[asset]
        df = yf.download(
            symbol,
            period=period,
            interval=time_frame,
        )

        if df.empty:
            print(f"No data for {asset}. Check ticker symbol or timeframe.")
            return pd.DataFrame()

        df.reset_index(inplace=True)
        return df

    except Exception as e:
        print(f"Error fetching data for {asset}: {e}")
        return pd.DataFrame()


def mass_import(asset_index, time_frame):

    asset_list = list(assets.keys())
    asset = asset_list[asset_index]

    frame_M15 = '15m'
    frame_M30 = '30m'
    frame_H1 = '1h'
    frame_H4 = '4h'
    frame_D1 = '1d'
    frame_W1 = '1wk'

    if time_frame == '1WK':
        data = get_quotes(frame_W1, '10y', asset=asset)
    elif time_frame == 'D1':
        data = get_quotes(frame_D1, '5y', asset=asset)
    elif time_frame == '4H':
        data = get_quotes(frame_H4, '2y', asset=asset)
    elif time_frame == '1H':
        data = get_quotes(frame_H1, '2y', asset=asset)
    elif time_frame == 'M30':
        data = get_quotes(frame_M30, '60d', asset=asset)
    elif time_frame == 'M15':
        data = get_quotes(frame_M15, '60d', asset=asset)
    else:
        data = get_quotes(frame_D1, 'D1', asset=asset)

    if data.empty:
        print(f"No data found for {asset} on timeframe {time_frame}")
        return np.array([])

    return data


def detect_key_levels(df, current_candle, backcandles=50, test_candles=10):
    """
    Detects key support and resistance levels in a given backcandles window.
    
    A level is identified if a candle's high is the highest or its low is the lowest 
    compared to `test_candles` before and after it.

    Parameters:
        df (pd.DataFrame): DataFrame containing 'High' and 'Low' columns.
        current_candle (int): The index of the current candle (latest available candle).
        backcandles (int): Number of candles to look back.
        test_candles (int): Number of candles before and after each candle to check.

    Returns:
        dict: A dictionary with detected 'support' and 'resistance' levels.
    """
    key_levels = {"support": [], "resistance": []}

    # Define the last candle that can be tested to avoid lookahead bias
    last_testable_candle = current_candle - test_candles

    # Ensure we have enough data
    if last_testable_candle < backcandles + test_candles:
        return key_levels  # Not enough historical data

    # Iterate through the backcandles window
    for i in range(current_candle - backcandles, last_testable_candle):
        high = df['High'].iloc[i]
        low = df['Low'].iloc[i]

        # Get surrounding window of test_candles before and after
        before = df.iloc[max(0, i - test_candles):i]
        after = df.iloc[i + 1: min(len(df), i + test_candles + 1)]

        # Check if current high is the highest among before & after candles
        if high > before['High'].max() and high > after['High'].max():
            key_levels["resistance"].append((i, high))

        # Check if current low is the lowest among before & after candles
        if low < before['Low'].min() and low < after['Low'].min():
            key_levels["support"].append((i, low))

    return key_levels


def fill_key_levels(df, backcandles=50, test_candles=10):
    """
    Adds a 'key_levels' column to the DataFrame where each row contains all
    key support and resistance levels detected up to that candle (including
    both the level value and the index of the candle that generated it).
    
    Parameters:
        df (pd.DataFrame): DataFrame containing 'High' and 'Low' columns.
        backcandles (int): Lookback window for detecting key levels.
        test_candles (int): Number of candles before/after for validation.

    Returns:
        pd.DataFrame: Updated DataFrame with the new 'key_levels' column.
    """
    df["key_levels"] = None  # Initialize the column

    from tqdm import tqdm
    for current_candle in tqdm(range(backcandles + test_candles, len(df))):
        # Detect key levels for the current candle
        key_levels = detect_key_levels(
            df, current_candle, backcandles, test_candles)

        # Collect support and resistance levels (with their indices) up to current_candle
        support_levels = [(idx, level) for (idx, level) in key_levels["support"]
                          if idx < current_candle]
        resistance_levels = [(idx, level) for (idx, level) in key_levels["resistance"]
                             if idx < current_candle]

        # Store the levels along with the originating candle index
        if support_levels or resistance_levels:
            df.at[current_candle, "key_levels"] = {
                "support": support_levels,
                "resistance": resistance_levels
            }

    return df


def flatten_yf_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Removes the 'Ticker' row level from a Yahoo Finance DataFrame
    that has multi-index columns like ('EURUSD=X', 'Open').

    Returns a DataFrame with simple column names like 'Open', 'High', etc.
    """
    # If columns are MultiIndex (e.g. ('EURUSD=X', 'Open')), flatten them
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(1)
    return df

def detect_fvg(data, lookback_period=14, body_multiplier=1.5):
    """
    Detects Fair Value Gaps (FVGs) in historical price data.

    Rules Added:
    - Middle candle body must be > avg body * body_multiplier
    - The FVG gap must be > 10% of the middle candle body
    """

    fvg_list = [None, None]

    for i in range(2, len(data)):
        first_high = data['High'].iloc[i-2]
        first_low = data['Low'].iloc[i-2]
        middle_open = data['Open'].iloc[i-1]
        middle_close = data['Close'].iloc[i-1]
        third_low = data['Low'].iloc[i]
        third_high = data['High'].iloc[i]

        # Middle candle body
        middle_body = abs(middle_close - middle_open)

        # Avg body over lookback
        prev_bodies = (data['Close'].iloc[max(0, i-1-lookback_period):i-1] -
                       data['Open'].iloc[max(0, i-1-lookback_period):i-1]).abs()
        avg_body_size = prev_bodies.mean()
        avg_body_size = avg_body_size if avg_body_size > 0 else 0.001

        # Minimum required gap size (10% of middle body)
        min_gap = middle_body * 0.10

        # -----------------------------
        # Check Bullish FVG
        # -----------------------------
        if third_low > first_high:
            gap_size = third_low - first_high

            if (middle_body > avg_body_size * body_multiplier
                    and gap_size > min_gap):

                fvg_list.append(('bullish', first_high, third_low, i))
                # print("bullish", first_high, third_low, i)
                continue

        # -----------------------------
        # Check Bearish FVG
        # -----------------------------
        if third_high < first_low:
            gap_size = first_low - third_high

            if (middle_body > avg_body_size * body_multiplier
                    and gap_size > min_gap):

                fvg_list.append(('bearish', first_low, third_high, i))
                # print("bearish", first_low, third_high, i)
                continue

        # No FVG
        fvg_list.append(None)

    return fvg_list

In [10]:
dataF = mass_import(0, '1H')
dataF = dataF.reset_index()            # Move Datetime from index to column
dataF.rename(columns={'Date': 'Gmt time'}, inplace=True)


try:
    my_data = dataF[['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']]
except:
    my_data = dataF[['Gmt time', 'Open', 'High', 'Low', 'Close', 'Volume']]


my_data = flatten_yf_columns(my_data)
my_data

/var/folders/pp/x__1tf4n7j98nw_8bwlmgxj00000gn/T/ipykernel_17257/15209537.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['EURUSD=X']: Timeout('Failed to perform, curl: (28) Operation timed out after 10000 milliseconds with 46662 bytes received. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


No data for EURUSD. Check ticker symbol or timeframe.
No data found for EURUSD on timeframe 1H


AttributeError: 'numpy.ndarray' object has no attribute 'reset_index'

In [11]:

my_data['FVG'] = detect_fvg(my_data)
my_data = fill_key_levels(my_data, backcandles=50, test_candles=10)
my_data.tail(20)

100%|██████████| 12314/12314 [00:24<00:00, 492.85it/s]


Price,Datetime,Open,High,Low,Close,Volume,FVG,key_levels
12354,2025-11-18 22:00:00+00:00,1.158614,1.159017,1.158078,1.158346,0,None,None
12355,2025-11-18 23:00:00+00:00,1.158346,1.158480,1.158078,1.158346,0,None,None
12356,2025-11-19 00:00:00+00:00,1.158346,1.158749,1.158212,1.158614,0,None,None
12357,2025-11-19 01:00:00+00:00,1.158614,1.158749,1.157943,1.158078,0,None,None
12358,2025-11-19 02:00:00+00:00,1.157943,1.158480,1.157675,1.158212,0,None,None
12359,2025-11-19 03:00:00+00:00,1.158212,1.158749,1.158078,1.158614,0,None,None
12360,2025-11-19 04:00:00+00:00,1.158749,1.158749,1.158078,1.158749,0,None,"{'support': [(12349, 1.1575413942337036)], 're..."
12361,2025-11-19 05:00:00+00:00,1.158883,1.159286,1.158749,1.159152,0,None,"{'support': [(12349, 1.1575413942337036)], 're..."
12362,2025-11-19 06:00:00+00:00,1.159286,1.159555,1.159017,1.159017,0,None,"{'support': [(12349, 1.1575413942337036)], 're..."
12363,2025-11-19 07:00:00+00:00,1.158883,1.159958,1.158078,1.158346,0,None,"{'support': [(12349, 1.1575413942337036)], 're..."


In [12]:
def plot_fvg_and_key_levels(df, start_idx, end_idx, extension=30):
    """
    Plots candlesticks, FVG zones, and key levels (support/resistance) for a
    subset of a DataFrame from `start_idx` to `end_idx`.
    
    The FVG column is expected to have tuples of the form:
        (fvg_type, start_price, end_price, trigger_index)

    The key_levels column is expected to have dictionaries of the form:
        {
          "support": [(idx, price), (idx, price), ...],
          "resistance": [(idx, price), (idx, price), ...]
        }

    Parameters:
    -----------
    df : pd.DataFrame
        Must contain: "Open", "High", "Low", "Close", "FVG", "key_levels".
    start_idx : int
        Starting row index for plotting.
    end_idx : int
        Ending row index for plotting.
    extension : int
        How far (in x-axis units/index steps) to extend the FVG rectangles
        and key-level lines.
    
    Returns:
    --------
    fig : plotly.graph_objects.Figure
        A Plotly Figure with the candlesticks, FVG, and key-level lines.
    """

    # Slice the DataFrame to the desired plotting range
    dfpl = df.loc[start_idx:end_idx]

    # Create the figure
    fig = go.Figure()

    # -- 1) Add Candlestick Chart --
    fig.add_trace(go.Candlestick(
        x=dfpl.index,
        open=dfpl["Open"],
        high=dfpl["High"],
        low=dfpl["Low"],
        close=dfpl["Close"],
        name="Candles"
    ))

    # -- 2) Add FVG Zones --
    for i, row in dfpl.iterrows():
        # Check if "FVG" is a valid tuple: (fvg_type, start_price, end_price, trigger_index)
        if isinstance(row.get("FVG"), tuple):
            fvg_type, start_price, end_price, trigger_idx = row["FVG"]

            # Choose a fill color based on bullish vs. bearish
            if fvg_type == "bullish":
                color = "rgba(0, 255, 0, 0.3)"   # greenish
            else:
                color = "rgba(255, 0, 0, 0.3)"   # reddish

            fig.add_shape(
                type="rect",
                x0=trigger_idx,
                x1=trigger_idx + extension,
                y0=start_price,
                y1=end_price,
                fillcolor=color,
                opacity=0.4,
                layer="below",
                line=dict(width=0),
            )

    # -- 3) Add Key Levels as Horizontal Lines --
    for i, row in dfpl.iterrows():
        key_levels = row.get("key_levels", None)
        if key_levels:
            # key_levels is a dict: {"support": [(idx, val), ...], "resistance": [(idx, val), ...]}
            support_levels = key_levels.get("support", [])
            resistance_levels = key_levels.get("resistance", [])

            # Plot support levels
            for (gen_idx, s_price) in support_levels:
                # We only draw the line if gen_idx is in (start_idx, end_idx)
                # You can decide to relax/omit this check if you want lines from outside the window.
                if start_idx <= gen_idx <= end_idx:
                    fig.add_shape(
                        type="line",
                        x0=gen_idx,
                        x1=gen_idx + extension,
                        y0=s_price,
                        y1=s_price,
                        line=dict(color="blue", width=2),
                        layer="below"
                    )

            # Plot resistance levels
            for (gen_idx, r_price) in resistance_levels:
                if start_idx <= gen_idx <= end_idx:
                    fig.add_shape(
                        type="line",
                        x0=gen_idx,
                        x1=gen_idx + extension,
                        y0=r_price,
                        y1=r_price,
                        line=dict(color="orange", width=2),
                        layer="below"
                    )

    # -- 4) Figure Aesthetics --
    fig.update_layout(
        width=1200,
        height=800,
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=False),
        plot_bgcolor='black',
        paper_bgcolor='black'
    )

    fig.show()
    return fig


fig = plot_fvg_and_key_levels(my_data, start_idx=12173, end_idx=12373, extension=30)